In [ ]:
#https://colab.research.google.com/notebooks/forms.ipynb#scrollTo=h9aZYKhly2h_

In [1]:
#@title important libraries
import json
import math
import pandas as pd
import os
import pickle
import joblib
from google.colab import output
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import sklearn
from sklearn import linear_model 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [2]:
#@title user_check
auth= json.load(open("authentication.json"))

user_name = 'nakul' #@param {type:"string"}
output.clear()
if user_name in auth:
  pass_key = 'n@123' #@param {type:"string"}
  #pass_key = input("passkey ")
  output.clear()
  if pass_key == auth[user_name]:
    info={}
    info["meta data"]={}
    info["meta data"]["user"]=user_name
    print("credential validated, please proceed for experiment")
  else:
    print("wrong passkey, try again")
else:
  print("user_name doest exist")

credential validated, please proceed for experiment


In [4]:
#@title Input Data
experiment_name = 'trial_1' #@param {type:"string"}
#experiment_name=input("please_give_experiment_name ")
os.mkdir(experiment_name)
output.clear()

csv_path = 'example.csv' #@param {type:"string"}
#csv_path = input("please provide input csv ")
output.clear()
try:
  data=pd.read_csv(csv_path)
  info["input_data"]={}
  info["input_data"]["input_csv_path"]=csv_path

  print("Columns in input csv are ", list(data.columns))
  print("length is ", len(data))
  info["input_data"]["length of data"]=len(data)
  data.to_csv(experiment_name+"/Raw_input_data.csv")
except:
  print("please enter correct path")

Columns in input csv are  ['Unnamed: 0', 'year', 'month', 'interest_rate', 'unemployment_rate', 'index_price']
length is  24


In [5]:
#@title Data Preprocessing
column_drop= 'yes' #@param ["no", "yes"]
#response=input("any columns to be dropped, y or n ? ")
output.clear()

try:
  if column_drop=="yes":
    inp=input("please enter comma separated column names to removed ")
    output.clear()
    inp=inp.split(",")
    data=data.drop(inp, axis=1)
    info["data_processing"]={}
    info["data_processing"]["columns_removed"] = inp
  else:
    pass
except:
  print("column doesnt exsist")

check_nan =data.isnull().values.any()
# printing the result

if check_nan:
  response=input("there are nan values in you input data, needs to be dropped before model training, shall we proceed ? ")
  output.clear()
  if response.lower()=="y":
      initial_l=len(data)
      data = data.dropna()
      print("after dropping the nan values, count of row have reduced from ", initial_l, "to", len(data))
      info["data_processing"]["nans_dropped"]= len(data)-initial_l



In [ ]:
#@title feature_and_label_selection
target = input("which is the dependent column ? ")
info["label_column_name"] = target
y=data[target]
columns = list(data.columns)
columns.remove(target)
os.mkdir(experiment_name+"/plots")
for column in columns:
  plt.scatter(y, data[column], color='red')
  plt.title( target +' Vs '+ column, fontsize=14)
  plt.xlabel(column, fontsize=14)
  plt.ylabel(target, fontsize=14)
  plt.grid(True)
  
  plt.savefig(experiment_name+"/plots/"+column+"_vs_"+target+'.png', bbox_inches='tight')
  plt.show()

In [7]:
#@title select_feature
feature_columns=input("which columns would you prefer as input ( comma separeted input plz) ? ")
feature_columns=feature_columns.split(",")
info["features_selected"] = feature_columns

x=data[feature_columns]

which columns would you prefer as input ( comma separeted input plz) ? interest_rate,unemployment_rate


In [9]:
#@title train_test_split
test_split_ratio = 0.2 #@param {type:"slider", min:0.1, max:1, step:0.1}

#ratio=float(input("how much subset for testing (0 to 1) ? "))
output.clear()
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_split_ratio, random_state=42)

print("Training set length is", len(X_train)," and testing set length is ", len(X_test))
info["training_split_ratio"] =  test_split_ratio
info["training_length"] =  len(X_train)
info["testing_length"] = len(X_test)

Training set length is 19  and testing set length is  5


In [10]:
#@title Training
# with sklearn
print("---------------------sklearn model-------------------------")
model_1 = linear_model.LinearRegression()
model_1.fit(X_train, y_train)

print('Intercept: \n', model_1.intercept_)
print('Coefficients: \n', model_1.coef_)

print("---------------------statsmodels model---------------------")
# with statsmodels
x = sm.add_constant(X_train) # adding a constant
 
model_2 = sm.OLS(y_train, x).fit()
 
print_model = model_2.summary()
print(print_model)


---------------------sklearn model-------------------------
Intercept: 
 2601.599844115359
Coefficients: 
 [ 260.26126267 -360.55339049]
---------------------statsmodels model---------------------
                            OLS Regression Results                            
Dep. Variable:            index_price   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     75.80
Date:                Thu, 08 Dec 2022   Prob (F-statistic):           6.90e-09
Time:                        17:41:20   Log-Likelihood:                -105.94
No. Observations:                  19   AIC:                             217.9
Df Residuals:                      16   BIC:                             220.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
             

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.8/dist-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [11]:
#@title Model_Testing
pred_1=model_1.predict(X_test)

x = sm.add_constant(X_test)
pred_2=model_2.predict(x)


mse_1 = sklearn.metrics.mean_squared_error(y_test, pred_1)
rmse_1 = math.sqrt(mse_1)
mse_2 = sklearn.metrics.mean_squared_error(y_test, pred_2)
rmse_2 = math.sqrt(mse_2)

info["result"]={}
info["result"]["errors"]={}
info["result"]["errors"]["sklearn_linear_regression"] = rmse_1
info["result"]["errors"]["statsmodels.api_regression"] = rmse_2

print("sklearn_linear_regression_error = ", rmse_1)
print("statsmodels.api_regression_error = ", rmse_2)



sklearn_linear_regression_error =  83.40926389351989
statsmodels.api_regression_error =  83.40926389352019


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [12]:
#@title input_data_report_model_saving
X_train["status"]="Train"
X_train=pd.concat([X_train, y_train], axis=1)
X_test["status"]="Test"
X_test=pd.concat([X_test, y_test], axis=1)
csv_used_for_training=X_train.append(X_test)
csv_used_for_training.to_csv(experiment_name+"/Processed_Data.csv",index=False)

data.to_csv()
with open(experiment_name + "/report.json", "w") as outfile:
    json.dump(info, outfile,indent=4)
joblib.dump(model_1,experiment_name+"/sklearn_linear_regression.pkl")
joblib.dump(model_2,experiment_name+"/statsmodels.pkl")

['trial_1/statsmodels.pkl']

In [13]:
#@title loading model
exp_name= input("enter experiment_name plz ")
path = exp_name
dir_list = os.listdir(path)
models=[]
for i in dir_list:
  if ".pkl" in i:
      models.append(i)
output.clear()
print("models available are ",models)
saved_model=input("model_name ? ")
output.clear()
# Load the pickled model
model = joblib.load(exp_name+"/"+saved_model)
about_model=json.load(open(exp_name+"/"+"report.json"))
print(about_model)

{'meta data': {'user': 'nakul'}, 'input_data': {'input_csv_path': 'example.csv', 'length of data': 24}, 'data_processing': {'columns_removed': ['Unnamed: 0', 'year']}, 'label_column_name': 'index_price', 'features_selected': ['interest_rate', 'unemployment_rate'], 'training_split_ratio': 0.2, 'training_length': 19, 'testing_length': 5, 'result': {'errors': {'sklearn_linear_regression': 83.40926389351989, 'statsmodels.api_regression': 83.40926389352019}}}


In [14]:
#@title Inference
#test_out_yourself
columns_needed=about_model["features_selected"]
input_data={}
for col in columns_needed:
  a=int(input(col))
  input_data[col]=[a]
sample=pd.DataFrame(input_data)
model.predict(sample)

interest_rate12
unemployment_rate6


array([3561.41465316])